In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import date
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from dateutil.relativedelta import relativedelta
import os
import itertools as it
import _pickle as cPickle
import gc
import sys
from statistics import mode

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

old_err_state = np.seterr(divide='raise')
ignored_states = np.seterr(**old_err_state)

In [2]:
def load_cpickle_gc(filename):
    file_Name = filename
    output = open(file_Name, 'rb')

    gc.disable()

    mydict = cPickle.load(output)

    gc.enable()
    output.close()
    return mydict

In [3]:
LAMBDA = load_cpickle_gc('/Users/yiliwang/Documents/Berkeley MFE/AFP/lambda estimation/lambda/LAMBDA_weekly_BAC')

In [71]:
NEWS = load_cpickle_gc('/Users/yiliwang/Documents/Berkeley MFE/AFP/lambda estimation/news dic/news_data')

### Convert News into Binary DF with Count Threshold

In [72]:
def binary_news(df, threshold=50):
    df_after = df
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            df_after.iloc[i,j] = 1 if df.iloc[i,j]>=threshold else 0
    return df_after

In [208]:
daily_news_df = pd.DataFrame(NEWS['BAC'].sum(axis=1))
daily_news_df = daily_news_df.rename(columns={0:"daily news count"})
NEWS_binary_df = binary_news(daily_news_df, threshold=100)

In [210]:
LAMBDA.keys()

dict_keys(['2016-01-04', '2016-01-11', '2016-01-19', '2016-01-26', '2016-02-02', '2016-02-09', '2016-02-17', '2016-02-24', '2016-03-02', '2016-03-09', '2016-03-16', '2016-03-23', '2016-03-31', '2016-04-07', '2016-04-14', '2016-04-21', '2016-04-28', '2016-05-05', '2016-05-12', '2016-05-19', '2016-05-26', '2016-06-03', '2016-06-10', '2016-06-17', '2016-06-24', '2016-07-01', '2016-07-11', '2016-07-18', '2016-07-25', '2016-08-01', '2016-08-08', '2016-08-15', '2016-08-22', '2016-08-29', '2016-09-06', '2016-09-13', '2016-09-20', '2016-09-27', '2016-10-04', '2016-10-12', '2016-10-19', '2016-10-26', '2016-11-02', '2016-11-09', '2016-11-17', '2016-11-25', '2016-12-02', '2016-12-09', '2016-12-16', '2016-12-23'])

In [215]:
NEWS_binary_df.head()

,daily news count
2016-01-05,0.0
2016-01-06,1.0
2016-01-07,1.0
2016-01-08,1.0
2016-01-09,0.0


In [216]:
def newsAdjustedLambda_generator(news_binary_df, LAMBDA_dic):
    adjusted_dic = {}
    news_binary_df.index = [str(ind)[:10] for ind in news_binary_df.index]
    
    for date in list(LAMBDA_dic.keys()):
        try:
            if int(news_binary_df.T[date]) == 1:
                adjusted_dic[str(date)[:10]] = LAMBDA_dic[str(date)[:10]]
        except:
            print("Date Missing in News Data: ", date)
    return adjusted_dic

In [217]:
newsAdjustedLambda_dic = newsAdjustedLambda_generator(NEWS_binary_df, LAMBDA)

Date Missing in News Data:  2016-01-04


In [218]:
newsAdjustedLambda_dic.keys() # filtered lambda dictionary with news

dict_keys(['2016-01-11', '2016-01-19', '2016-02-02', '2016-02-09', '2016-03-16', '2016-03-31', '2016-04-07', '2016-04-14', '2016-04-28', '2016-05-19', '2016-06-03', '2016-06-10', '2016-06-24', '2016-07-11', '2016-07-18', '2016-08-29', '2016-09-27', '2016-10-04', '2016-10-19', '2016-10-26', '2016-11-09', '2016-11-17', '2016-12-02', '2016-12-09', '2016-12-16'])